In [80]:
# 导入相关的依赖包
import numpy as np
import gzip
import os 
import os.path 
import shutil 
import scipy.io as scio

In [15]:
# 本段代码用于将训练集和测试集中不同刺激情况下的数据分开
# 刺激的类型包括以下的三种类型   '# S1 obj '       '# S2 match '       '# S2 nomatch'

# 设置原始路径（对应训练集还是测试集）
path = 'C:\\Users\\Administrator\\Desktop\\xujiu\\SMNI_CMI_TEST\\SMNI_CMI_TEST\\'
# 读取路径下的的所有二十个实验对象（包括十个正对象以及十个负对象）
all_sub_folder = os.listdir(path)

# 对于每个实验对象的文件夹进行遍历操作（每个对象文件夹下面都有三十个实验）   
# PS:对于同一个实验对象，在每次的实验中对应的刺激可能不同
for i in range(len(all_sub_folder)):
    sub_folder = all_sub_folder[i]  
    whole_path = path + sub_folder 
# 对于某一个实验对象下的三十次实验样本进行遍历
    all_sub_file = os.listdir(whole_path)
    for j in range(len(all_sub_file)):
# 获得某一个实验对象在某一次实验下的数据文件路径
        new_path = whole_path + '\\' + all_sub_file[j]
# 对数据文件进行读取，解码
        fp = gzip.open(new_path)
        contents = fp.read()
        fp.close()
        u_str = contents.decode('utf-8') # u_str 现在是unicode 字符串
# 对解码后的数据文件（字符串）进行拆分，同时赋值给data变量
        data = u_str.split('\n')
# 对于变量data中的第3个字符串（Python中从0开始计数）存在着刺激类型的字段    
        matching_conditions = data[3]
        matching_conditions = matching_conditions.split(',')
# 对表示刺激类型的字符串进行判别，完成文件的归类拷贝        
        if matching_conditions[0] == '# S2 nomatch':
            targetDir = 'C:\\Users\\Administrator\\Desktop\\xujiu\\test\\S2 nomatch'
            shutil.copy(new_path,  targetDir)

20


In [84]:
# 此段代码用于从归类之后的拷贝文件中获取脑电数据以及对应实验对象的标签

# 创建数组S1_test_value，用于存放数据
# 第一维为实验样本的个数（200），第二维是单个实验样本的信道数（64），第三维是单个实验样本的长度
S1_test_value = np.ones((200,64,256), dtype = np.float64)  

# 创建数组S1_test_label，记录样本的标签
S1_test_label = np.ones((200,1), dtype = np.float64)
# 对应类别数据的路径，同时获得该路径下的所有实验样本
S1_test_path = 'C:\\Users\\Administrator\\Desktop\\xujiu\\test\\S1 obj\\'
S1_test = os.listdir(S1_test_path)

# 遍历所有实验样本
for i in range(len(S1_test)):
    trial_data_path = S1_test_path + S1_test[i]
# 进行样本标签判别，同时存入变量S1_test_label
    if (S1_test[i])[3] == 'c':
        S1_test_label[i] = 0
# 读取单个实验样本
    fp = gzip.open(trial_data_path)
    contents = fp.read()
    fp.close()
    u_str = contents.decode('utf-8') # u_str 现在是unicode 字符串
    trial_data = u_str.split('\n')
# 获得实验（Trial）号，用来筛选出数据
    number_trial = trial_data[3].split(' ')[-1]
# 定义一个空变量object_data，存放筛选出来的数据（刨去其他元数据）
    object_data = []
    for j in range(len(trial_data)):
        if trial_data[j].split(' ')[0] == number_trial:
            object_data.append(trial_data[j])
# 对于变量object_data进行遍历，依次获得样本值data_value
    for k in range(len(object_data)):
        data_value = float(object_data[k].split(' ')[-1])
# 将样本值data_value，对应填入S1_test_value
        S1_test_value[i,k//256,k%256] = data_value
    
# 将数据变量S1_test_value，输出为mat格式文件
mat_data_path = S1_test_path + 'S1_test_value.mat'
scio.savemat(mat_data_path, {'S1_test_value':S1_test_value}) 

# 将标签变量S1_test_label，输出为mat格式文件
mat_data_path = S1_test_path + 'S1_test_label.mat'
scio.savemat(mat_data_path, {'S1_test_label':S1_test_label})  